In [ ]:
import digitalhub as dh

proj = dh.get_or_create_project("project-container")

In [ ]:
# Run container job

func_cont = proj.new_function(name="function-container",
                              kind="container",
                              image="hello-world:latest")
run_cont = func_cont.run("job")

In [ ]:
# Build a container

# First compress and upload the code to S3
import os
from pathlib import Path
from zipfile import ZipFile

code_zip = "streamlit-code.zip"
with ZipFile(code_zip, 'w') as f:
    for root, _, files in os.walk("streamlit-code"):
        for file in files:
            file_path = Path(root, file)
            f.write(file_path, file)

remote_path = "s3://datalake/streamlit-code.zip"
art = proj.new_artifact(name="streamlit-code",
                        kind="artifact",
                        path=remote_path,
                        src_path=code_zip)
art.upload()


# Create function and run
function = proj.new_function(name="build",
                             kind="container",
                             base_image="python:3.9-slim",
                             command="streamlit",
                             args=["run", "app.py", "--server.fileWatcherType=none"])

In [ ]:
# Build the container
run_build = function.run(action="build",
                         context_refs=[{
                             "destination": code_zip,
                             "protocol": "zip+s3",
                             "source": remote_path
                             }],
                         instructions=[
                             "apt-get update && apt-get install -y build-essential curl software-properties-common git",
                             "rm -rf var/lib/apt/lists/*",
                             "pip3 install -r requirements.txt"
                             ])

In [ ]:
# Wait until run is completed
run_build.refresh()

In [ ]:
# Serve the streamlit container
run_serve = function.run("serve",
                         service_ports= [{"port": 8085, "target_port": 8501}],
                         service_type="ClusterIP")

# To access the streamlit container, you need to espose the service with KRM and API Gatway